# ImageNet Transfer Learning Using ResNet50

### Implemented by Pratham Shah, 240905614, for the Cryptonite Research Taskphase

## Imports

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Add, Dense, Flatten, GlobalAveragePooling2D, MaxPooling2D, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetV2S
import shutil
from tensorflow.keras.metrics import TopKCategoricalAccuracy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.models import load_model
from sklearn.calibration import label_binarize
from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, auc, roc_curve, log_loss, accuracy_score, precision_score, f1_score, recall_score

2025-06-24 12:48:06.044588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750769286.066768     483 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750769286.073554     483 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Consolidating the 4 train subsets into 1

In [2]:
input_root = "/kaggle/input/imagenet100"
train_output = "/kaggle/working/imagenet100/train_combined"
val_path = "/kaggle/input/imagenet100/val.X" 

os.makedirs(train_output, exist_ok=True)

for i in range(1, 5):
    subset_path = f"{input_root}/train.X{i}"

    for class_name in os.listdir(subset_path):
        src = f"{subset_path}/{class_name}"
        dst = f"{train_output}/{class_name}"

        if not os.path.exists(dst):
            shutil.copytree(src, dst)
            print(f"Copied: {class_name} from train.X{i}")
        else:
            print(f"Skipped: {class_name} already exists")

Skipped: n01531178 already exists
Skipped: n01440764 already exists
Skipped: n01494475 already exists
Skipped: n01950731 already exists
Skipped: n01795545 already exists
Skipped: n01632777 already exists
Skipped: n02012849 already exists
Skipped: n01775062 already exists
Skipped: n02007558 already exists
Skipped: n01484850 already exists
Skipped: n01930112 already exists
Skipped: n01984695 already exists
Skipped: n02037110 already exists
Skipped: n02018795 already exists
Skipped: n01695060 already exists
Skipped: n01978455 already exists
Skipped: n01749939 already exists
Skipped: n01824575 already exists
Skipped: n01914609 already exists
Skipped: n01833805 already exists
Skipped: n01924916 already exists
Skipped: n01665541 already exists
Skipped: n01820546 already exists
Skipped: n01687978 already exists
Skipped: n01818515 already exists
Skipped: n02058221 already exists
Skipped: n01677366 already exists
Skipped: n02077923 already exists
Skipped: n01698640 already exists
Skipped: n0159

## Image Augmentation and Batch-wise generation

In [3]:
augmentor = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet_v2.preprocess_input,
    rotation_range = 30,
    zoom_range = 0.2,
    shear_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

train_gen = augmentor.flow_from_directory("/kaggle/working/imagenet100/train_combined", batch_size=32, target_size=(256, 256),)
val_gen = augmentor.flow_from_directory("/kaggle/input/imagenet100/val.X", batch_size=32, shuffle=False, target_size=(256, 256),)

Found 130000 images belonging to 100 classes.
Found 5000 images belonging to 100 classes.


## The Model

In [4]:
base = EfficientNetV2S(input_shape=(256, 256, 3), include_top=False)
base.trainable = True

model = Sequential([
    base,
    BatchNormalization(),
    GlobalAveragePooling2D(),
    
    Dense(512, activation='relu', kernel_regularizer=l2(2e-4)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(256, activation='relu', kernel_regularizer=l2(2e-4)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(100, activation='softmax', kernel_regularizer=l2(2e-4))
])

optimiser = AdamW(learning_rate=2e-4, weight_decay=2e-5)

model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.15),
    optimizer = optimiser,
    metrics = ["accuracy", TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
)

model.summary()

I0000 00:00:1750769291.878592     483 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-s (Functional)        │ (None, 8, 8, 1280)          │      20,331,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 8, 8, 1280)          │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 100)                 │          25,700 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,152,452 (80.69 MB)

 Trainable params: 20,994,484 (80.09 MB)

 Non-trainable params: 157,968 (617.06 KB)

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=5,
    restore_best_weights=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

saver = ModelCheckpoint(
    "/kaggle/working/best_model.h5",
    monitor='val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=12,
    callbacks=[early_stop, reduce_lr, saver]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/12


I0000 00:00:1750769421.374829     547 service.cc:148] XLA service 0x7d40a40037c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750769421.374877     547 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750769432.527680     547 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1750769448.958111     547 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750769449.155144     547 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750769449.638440     547 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750769449.8460

2524/4063 ━━━━━━━━━━━━━━━━━━━━ 13:15 517ms/step - accuracy: 0.6899 - loss: 2.5256 - top_3_accuracy: 0.8067

E0000 00:00:1750770815.617861     548 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750770815.808670     548 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750770816.250946     548 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750770816.451896     548 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1750770816.991401     548 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:0

4063/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 537ms/step - accuracy: 0.7311 - loss: 2.3574 - top_3_accuracy: 0.8459
Epoch 1: val_accuracy improved from -inf to 0.85120, saving model to /kaggle/working/best_model.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 2515s 572ms/step - accuracy: 0.7312 - loss: 2.3573 - top_3_accuracy: 0.8460 - val_accuracy: 0.8512 - val_loss: 1.7996 - val_top_3_accuracy: 0.9500 - learning_rate: 2.0000e-04
Epoch 2/12
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 0s 527ms/step - accuracy: 0.8633 - loss: 1.7778 - top_3_accuracy: 0.9586
Epoch 2: val_accuracy improved from 0.85120 to 0.85560, saving model to /kaggle/working/best_model.h5
4063/4063 ━━━━━━━━━━━━━━━━━━━━ 2251s 554ms/step - accuracy: 0.8633 - loss: 1.7778 - top_3_accuracy: 0.9586 - val_accuracy: 0.8556 - val_loss: 1.7195 - val_top_3_accuracy: 0.9516 - learning_rate: 2.0000e-04
Epoch 3/12
4019/4063 ━━━━━━━━━━━━━━━━━━━━ 23s 524ms/step - accuracy: 0.8805 - loss: 1.6582 - top_3_accuracy: 0.9667
Epoch 3: val_accuracy did not improve from 0.85560
4

## Performance Monitoring

In [ ]:
tr = model.evaluate(train_gen)
te = model.evaluate(val_gen)
predictions = model.predict(val_gen)

In [ ]:
for item in tr, te:
    print(f"Accuracy: {item[1]}")
    print(f"loss: {item[0]}")

In [ ]:
print("example of the cnn output showing probabilities:")
print(predictions[0])

final = np.argmax(predictions, axis=1) #converts row into single value of highest prob. index = class ID
real_labels = real_labels = val_gen.classes

indexed_items = list(val_gen.class_indices.keys())

print(classification_report(real_labels, final, target_names = indexed_items)) #classification report and confusion matrix

cm = confusion_matrix(real_labels, final)
sns.heatmap(cm, annot=True, fmt='d', cmap="inferno")
plt.title("Confusion matrix")
plt.xticks(np.arange(100) + 0.5, indexed_items, rotation=90)
plt.yticks(np.arange(100) + 0.5, indexed_items, rotation=0)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
np.fill_diagonal(cm, 0) #removing all correctly classified examples - in the diagonal
sns.heatmap(cm, annot=True, cmap="coolwarm")
plt.xticks(np.arange(100) + 0.5, indexed_items, rotation=90)
plt.yticks(np.arange(100) + 0.5, indexed_items, rotation=0)
plt.title("Misclassification report")

In [ ]:
filenames = val_gen.filenames
errors = np.where(final != real_labels)[0]

for i in errors[:5]:
    # Load image using testing.directory + filenames[i]
    img_path = os.path.join(val_gen.directory, filenames[i])
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(256, 256))
    img_array = tf.keras.preprocessing.image.img_to_array(img).astype("uint8")
    
    plt.imshow(img_array.astype("uint8"))
    plt.title(f"True: {indexed_items[real_labels[i]]}, Pred: {indexed_items[final[i]]}")
    plt.axis("off")
    plt.show()

In [ ]:
print(f"Matthew's Correlation Coefficient : {matthews_corrcoef(real_labels, final)}")
print(f"Accuracy : {accuracy_score(real_labels, final)}")
print(f"f1 score : {f1_score(real_labels, final, average='weighted')}")
print(f"Log loss : {log_loss(real_labels, predictions)}")
print(f"Recall : {recall_score(real_labels, final, average='weighted')}")
print(f"Precision : {precision_score(real_labels, final, average='weighted')}")

In [ ]:
real_bin = label_binarize(real_labels, classes=np.arange(100))

plt.figure(figsize=(15, 10))

for i in range(100):
    
    false_p_rate, true_p_rate, thresholds = roc_curve(real_bin[:, i], predictions[:, i])
    
    roc_auc = auc(false_p_rate, true_p_rate) #higher area under curve - straighter the lines - lesser the FPR
    
    plt.plot(false_p_rate, true_p_rate, label=f"{indexed_items[i]} - AUC: {roc_auc:.2f}")

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curves (1 v/s rest)')
plt.legend(loc='lower right')
plt.grid()
plt.show()

## Conclusion